In [1]:
import tensorflow as tf
import numpy as np
import simplejson
import matplotlib.pyplot as plt
%matplotlib inline
import threading

import utils.train_utils as train_utils
import utils.googlenet_load as googlenet_load
import tensorflow.contrib.rnn as rnn
import time
import tensorflow.contrib.slim as slim
import string
from train import build_pred

In [2]:
import os
import subprocess
from scipy.misc import imread
from utils import train_utils
from utils.annolist import AnnotationLib as al
#from utils.stitch_wrapper import stitch_rects
from utils.train_utils import add_rectangles
from utils.rect import Rect
from evaluate import add_rectangles
import cv2

In [3]:
hypes = 'hypes/lstm_cnn.json'
with open(hypes, 'r') as f:
    H = simplejson.load(f)
image_height = H['image_height']
image_width = H['image_width']
H['batch_size'] = 1

In [4]:
tf.reset_default_graph()
x_in = tf.placeholder(dtype=tf.float32, shape=[image_height, image_width, 3])
pred_boxes, pred_logits, pred_confs = build_pred(H=H, phase='test', x_in=tf.expand_dims(x_in,0), reuse=False)

In [5]:
for op in tf.global_variables(): print op.name

InceptionV1/Conv2d_1a_7x7/weights:0
InceptionV1/Conv2d_1a_7x7/BatchNorm/beta:0
InceptionV1/Conv2d_1a_7x7/BatchNorm/gamma:0
InceptionV1/Conv2d_1a_7x7/BatchNorm/moving_mean:0
InceptionV1/Conv2d_1a_7x7/BatchNorm/moving_variance:0
InceptionV1/Conv2d_2b_1x1/weights:0
InceptionV1/Conv2d_2b_1x1/BatchNorm/beta:0
InceptionV1/Conv2d_2b_1x1/BatchNorm/gamma:0
InceptionV1/Conv2d_2b_1x1/BatchNorm/moving_mean:0
InceptionV1/Conv2d_2b_1x1/BatchNorm/moving_variance:0
InceptionV1/Conv2d_2c_3x3/weights:0
InceptionV1/Conv2d_2c_3x3/BatchNorm/beta:0
InceptionV1/Conv2d_2c_3x3/BatchNorm/gamma:0
InceptionV1/Conv2d_2c_3x3/BatchNorm/moving_mean:0
InceptionV1/Conv2d_2c_3x3/BatchNorm/moving_variance:0
InceptionV1/Mixed_3b/Branch_0/Conv2d_0a_1x1/weights:0
InceptionV1/Mixed_3b/Branch_0/Conv2d_0a_1x1/BatchNorm/beta:0
InceptionV1/Mixed_3b/Branch_0/Conv2d_0a_1x1/BatchNorm/gamma:0
InceptionV1/Mixed_3b/Branch_0/Conv2d_0a_1x1/BatchNorm/moving_mean:0
InceptionV1/Mixed_3b/Branch_0/Conv2d_0a_1x1/BatchNorm/moving_variance:0
In

In [8]:
saver = tf.train.Saver()
true_json = './data/brainwash/val_boxes.json'
true_annos = al.parse(true_json)

with tf.Session() as sess:
    saver.restore(sess, 'log_dir/checkpoint/-2900')
    annolist = al.AnnoList()
    import time; t = time.time()
    for i in range(0, 500):
        true_anno = true_annos[i]
        img = imread('./data/brainwash/%s' % true_anno.imageName)
        feed = {x_in: img}
        (np_pred_boxes, np_pred_confidences) = sess.run([pred_boxes, pred_confs], feed_dict=feed)
        pred_anno = al.Annotation()
        pred_anno.imageName = true_anno.imageName
        new_img, rects = add_rectangles(H, [img], np_pred_confidences, np_pred_boxes,
                                        use_stitching=True, rnn_len=H['rnn_len'], min_conf=0.7,
                                        show_suppressed=False)
    
        pred_anno.rects = rects
        annolist.append(pred_anno)

        if i % 10 == 0 and i < 200:
            pass
            fig = plt.figure(figsize=(12, 12))
            plt.imshow(new_img)
        if i % 100 == 0:
            print(i)
    avg_time = (time.time() - t) / (i + 1)
    print('%f images/sec' % (1. / avg_time))
    

INFO:tensorflow:Restoring parameters from log_dir/checkpoint/-2900


ImportError: dlopen(utils/stitch_wrapper.so, 2): Symbol not found: __Z12filter_rectsRKSt6vectorIS_IS_I4RectSaIS0_EESaIS2_EESaIS4_EEPS2_ffff
  Referenced from: utils/stitch_wrapper.so
  Expected in: flat namespace
 in utils/stitch_wrapper.so

In [7]:
annolist.

 ...]